In [ ]:
import re
import nltk
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
book = pd.read_csv(r"/content/drive/MyDrive/Book data/Books.csv")
rate = pd.read_csv(r"/content/drive/MyDrive/Book data/Ratings.csv")
user = pd.read_csv(r"/content/drive/MyDrive/Book data/Users.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
book.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [ ]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [ ]:
rate.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [ ]:
rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
user.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
print("Book:\n")
print(book.isna().sum())
print('-'*50)
print("User:\n")
print(user.isna().sum())
print('-'*50)
print("Rate:\n")
print(rate.isna().sum())

Book:

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
--------------------------------------------------
User:

User-ID          0
Location         0
Age         110762
dtype: int64
--------------------------------------------------
Rate:

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


# Popularity Based Recommendation System

Condition:

If more than 250 people have voted that book than we can recommend it.

In [ ]:
# merge rate and book dataframe based on ISBN column

books_rating = rate.merge(book,on='ISBN')
books_rating.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
# Book name and number of rating it got

num_rating_of_book = books_rating.groupby('Book-Title').count()['Book-Rating'].reset_index()

num_rating_of_book.rename(columns={"Book-Rating":"Num Of Rating"},inplace=True)

In [ ]:
num_rating_of_book

,Book-Title,Num Of Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [ ]:
# Book name and avg rating it got

avg_rating_of_book = books_rating.groupby('Book-Title').mean()['Book-Rating'].reset_index()

avg_rating_of_book.rename(columns={"Book-Rating":"Avg Of Rating"},inplace=True)

avg_rating_of_book

,Book-Title,Avg Of Rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [ ]:
# mergining of num_rating_of_book and avg_rating_of_book

popularity_df = num_rating_of_book.merge(avg_rating_of_book,on='Book-Title')
popularity_df

,Book-Title,Num Of Rating,Avg Of Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [ ]:
popularity_df = popularity_df[popularity_df['Num Of Rating']>=250].reset_index()
popularity_df

,index,Book-Title,Num Of Rating,Avg Of Rating
0,764,1984,284,4.454225
1,818,1st to Die: A Novel,509,3.575639
2,1048,2nd Chance,356,3.269663
3,1760,A Bend in the Road,346,3.364162
4,2281,"A Child Called \It\"": One Child's Courage to S...",265,4.086792
...,...,...,...,...
181,233850,White Oleander : A Novel,387,3.506460
182,233851,White Oleander : A Novel (Oprah's Book Club),356,3.772472
183,234740,Wicked: The Life and Times of the Wicked Witch...,326,3.766871
184,234951,Wild Animus,2502,1.019584


In [ ]:
popularity_df = popularity_df.iloc[:,1:]
popularity_df

,Book-Title,Num Of Rating,Avg Of Rating
0,1984,284,4.454225
1,1st to Die: A Novel,509,3.575639
2,2nd Chance,356,3.269663
3,A Bend in the Road,346,3.364162
4,"A Child Called \It\"": One Child's Courage to S...",265,4.086792
...,...,...,...
181,White Oleander : A Novel,387,3.506460
182,White Oleander : A Novel (Oprah's Book Club),356,3.772472
183,Wicked: The Life and Times of the Wicked Witch...,326,3.766871
184,Wild Animus,2502,1.019584


In [ ]:
popular_books = popularity_df.sort_values('Avg Of Rating',ascending=False).head(50)
popular_books

,Book-Title,Num Of Rating,Avg Of Rating
55,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
53,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
56,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
54,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
52,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
132,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
123,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
57,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
165,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
169,To Kill a Mockingbird,510,4.700000


In [ ]:
# take author name, publisher name, image url, num of rating, avg of rating

top_books_info = popular_books.merge(book, on='Book-Title').drop_duplicates('Book-Title').reset_index()
top_books_info = top_books_info.iloc[:,1:]
top_books_info

,Book-Title,Num Of Rating,Avg Of Rating,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0439136350,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,0439139597,J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...
2,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,0590353403,J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...
3,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,043935806X,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
4,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,0439064872,J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...
5,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117,0345339681,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...
6,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370,0345339703,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...
7,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652,059035342X,J. K. Rowling,1999,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...
8,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769,0345339711,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...
9,To Kill a Mockingbird,510,4.700000,0446310786,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...


In [ ]:
top_books_info.columns

Index(['Book-Title', 'Num Of Rating', 'Avg Of Rating', 'ISBN', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

In [ ]:
top_books_info = top_books_info[['Book-Title', 'Num Of Rating', 'Avg Of Rating','Book-Author','Image-URL-M',]]
top_books_info.head()

,Book-Title,Num Of Rating,Avg Of Rating,Book-Author,Image-URL-M
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...
2,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...
3,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...
4,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...


# Collaborative Filtering Recommendation System

Conditions:
- Target only the users who has rated more than 200 books
- Taking only those books who has 50 rating

(contains rating for each books by different users)

| Users/Books | User1 | User2 | ... | UserN |
| --- | --- | --- | --- | --- |
| Book1 | 5 | 2 | ... | 0 |
| Book2 | 0 | 1 | ... | 2 |
| Book3 | 3 | 5 | ... | 1 |
| Book4 | 2 | 0 | ... | 1 |

In [ ]:
books_rating.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
# get all the users who have voted for more than 200 books

bool_values = books_rating.groupby("User-ID").count()['Book-Rating']>200
get_index_of_users = bool_values[bool_values].index

In [ ]:
filtered_rating = books_rating[books_rating['User-ID'].isin(get_index_of_users)].reset_index().iloc[:,1:]
filtered_rating.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
# Take books who has more than 50 ratings

bool_values_50 = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
get_index_of_books = bool_values_50[bool_values_50].index

In [ ]:
filtered_rating = filtered_rating[filtered_rating['Book-Title'].isin(get_index_of_books)].reset_index().iloc[:,1:]
filtered_rating.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
1,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...


In [ ]:
filtered_rating.shape

(58586, 10)

In [ ]:
import copy
final_rating = copy.deepcopy(filtered_rating)  
final_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
1,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
2,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
4,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
58581,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...
58582,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA,http://images.amazon.com/images/P/0670809381.0...,http://images.amazon.com/images/P/0670809381.0...,http://images.amazon.com/images/P/0670809381.0...
58583,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...
58584,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...


In [ ]:
# make table which is mentioned in comment table

pt = final_rating.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace=True)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


706 Books & 810 Users

In [ ]:
# Using cosine similarity to recommend the books

from sklearn.metrics.pairwise import cosine_similarity

similarity_score = cosine_similarity(pt)
similarity_score

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [ ]:
# similarity of all the books with each other

similarity_score.shape

(706, 706)

In [ ]:
def recommendationSystem(book_name):
  values = []
  # fetch index of movies
  index = np.where(pt.index==book_name)[0][0]
  suggestions_scores = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1], reverse=True)[1:6]
  print(suggestions_scores)
  for i in suggestions_scores:
    values.append(pt.index[i[0]])
  return values

In [ ]:
for i in list(pt.index):
  if "harry potter" in str.lower(i):
    print(i)

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))


In [ ]:
recommendationSystem("Harry Potter and the Chamber of Secrets (Book 2)")

[(230, 0.6089508035980855), (228, 0.5706401143095313), (232, 0.47989087082211423), (231, 0.44748613765765954), (229, 0.35555787575044107)]


['Harry Potter and the Prisoner of Azkaban (Book 3)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 'Harry Potter and the Order of the Phoenix (Book 5)']

# Content Based Recommendation System

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Book data/courses.csv")
data.head(2)

,CourseId,CourseTitle,DurationInSeconds,ReleaseDate,Description,AssessmentStatus,IsCourseRetired
0,abts-advanced-topics,BizTalk 2006 Business Process Management,22198,2008-10-25,This course covers Business Process Management...,Live,no
1,abts-fundamentals,BizTalk 2006 Fundamentals,24305,2008-06-01,Despite the trend towards service-oriented arc...,Live,no


In [ ]:
courses = data[['CourseTitle','Description']]
courses

,CourseTitle,Description
0,BizTalk 2006 Business Process Management,This course covers Business Process Management...
1,BizTalk 2006 Fundamentals,Despite the trend towards service-oriented arc...
2,Agile Team Practices with Scrum,This course is much different than most of the...
3,Windows Server AppFabric Fundamentals,NaN
4,ASP.NET 3.5 Advanced Topics,This course covers more advanced topics in ASP...
...,...,...
8011,Mocking with Moq and NUnit,Writing effective unit tests can be tricky whe...
8012,Identity and Access Management on AWS: Users,At the core of Identity and Access Management ...
8013,Secure Coding: Using Components with Known Vul...,Do you know if old components you are using ar...
8014,Implementing Amazon S3 Storage on AWS,AWS S3 is one of the most fundamental services...


In [ ]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8016 entries, 0 to 8015
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CourseTitle  8016 non-null   object
 1   Description  8011 non-null   object
dtypes: object(2)
memory usage: 125.4+ KB


In [ ]:
courses = courses.dropna()
courses = courses.reset_index().iloc[:,1:]
courses.insert(0, column='ID',value=[i for i in range(len(courses))])
courses.head(2)

,ID,CourseTitle,Description
0,0,BizTalk 2006 Business Process Management,This course covers Business Process Management...
1,1,BizTalk 2006 Fundamentals,Despite the trend towards service-oriented arc...


In [ ]:
# No duplicated values
courses[courses.duplicated()]

,ID,CourseTitle,Description


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tf_idf = TfidfVectorizer(min_df = 3, max_features=None, strip_accents='unicode', analyzer='word', 
                         token_pattern='\w{1,}', ngram_range=(1,3), stop_words='english')


courses['Description'] = courses['Description'].fillna('')

# Creating into document matrix i.e vector of numbers
tf_matrix = tf_idf.fit_transform(courses['Description']) # create sparse matrix
tf_matrix

<8011x51200 sparse matrix of type '<class 'numpy.float64'>'
	with 689536 stored elements in Compressed Sparse Row format>

In [ ]:
tf_matrix.shape

(8011, 51200)

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

# compute the sigmoid kernel

sig = sigmoid_kernel(tf_matrix, tf_matrix)
sig

array([[0.76160236, 0.76159613, 0.76159457, ..., 0.76159428, 0.76159423,
        0.76159428],
       [0.76159613, 0.76160236, 0.76159429, ..., 0.76159425, 0.76159421,
        0.76159434],
       [0.76159457, 0.76159429, 0.76160236, ..., 0.76159446, 0.76159419,
        0.76159445],
       ...,
       [0.76159428, 0.76159425, 0.76159446, ..., 0.76160236, 0.76159429,
        0.7615943 ],
       [0.76159423, 0.76159421, 0.76159419, ..., 0.76159429, 0.76160236,
        0.76159502],
       [0.76159428, 0.76159434, 0.76159445, ..., 0.7615943 , 0.76159502,
        0.76160236]])

In [ ]:
# Reverse mapping of indices and course titles

indices = pd.Series(courses.index, index=courses['CourseTitle']).drop_duplicates()
indices

CourseTitle
BizTalk 2006 Business Process Management                         0
BizTalk 2006 Fundamentals                                        1
Agile Team Practices with Scrum                                  2
ASP.NET 3.5 Advanced Topics                                      3
ASP.NET Ajax Advanced Topics                                     4
                                                              ... 
Mocking with Moq and NUnit                                    8006
Identity and Access Management on AWS: Users                  8007
Secure Coding: Using Components with Known Vulnerabilities    8008
Implementing Amazon S3 Storage on AWS                         8009
Identity and Access Management on AWS: Roles and Groups       8010
Length: 8011, dtype: int64

In [ ]:
def recommendedCourses(title, sig=sig):
  # Get indices
  idx = indices[title]

  # Get the pairwise similarity scores
  sig_scores = list(enumerate(sig[idx]))

  # Sort the values
  sig_scores = sorted(sig_scores, key = lambda x:x[1], reverse=True)
  
  # Scores of 5 most similar courses
  sig_scores = sig_scores[1:6]

  # courses indices
  courses_indices = [i[0] for i in sig_scores]

  return courses['CourseTitle'].iloc[courses_indices]

In [ ]:
recommendedCourses("Mocking with Moq and NUnit")

6790    Mocking in .NET Core Unit Tests with Moq: Gett...
283                                      Mocking With Moq
6283               Advanced C++ Mocking Using Google Mock
274                              Rhino Mocks Fundamentals
7846            Introduction to .NET Testing with NUnit 3
Name: CourseTitle, dtype: object